In [33]:
import numpy as np

In [34]:
E       = 150000            # Tegangan Sistem (volt)
h       = 3                 # depth / kedalaman penanaman konduktor (m)
p       = 100               # tanahan jenis tanah (ohm-m)
ps      = 3000              # tahanan permukaan tanah (koral) (ohm-m)
hs      = 0.1               # ketebalan lapisan koral
a       = 0.012             # diameter pembumian Mesh (m)
b       = 0.012             # diameter konduktor pembumian Rod (m)
aa      = np.sqrt(a*2*h)    # (a') untuk konduktor yang ditanam pada kedalaman h
d       = b
D       = 10                # jarak antar konduktor grid
A       = 6000              # luas area sistem pembumian (m^2)
K1      = 0.1
K2      = 4.5               
Lr      = 3                 # panjang batang konduktor Rod (m)
nR      = 32                # jumlah konduktor batang Rod
LR      = 100               # panjang total konduktor Rod (m)
LC      = 660               # panjang total konduktor Mesh (m)

Vf      = 120               # tegangan rms ke tanah (volt)
Z1      = 4                 # Impedansi positif (ohm)
TCAP    = 3.42              # faktor kapasitansi panas konduktor (J/cm3/*C)
ts      = 1                 # lama gangguan (detik)
ar      = 0.00393           # koefisien  panas resistivitas (*C)
pr      = 1.72              # resistivitas konduktor pentanahan (ohm/cm)
Ko      = 234               # Konstanta (1/ar) (*C)
Tm      = 1083              # suhu maksimum konduktor (*C)
Ta      = 34                # suhu tanah sekitar

Kii     = 1
m       = 6                 # jumlah_konduktor_sisi_panjang
n       = 10                # jumlah_konduktor_sisi_lebar

In [35]:
# RG (Resistansi Pentanahan) Grid-Rod (ohm)
R1 = (p/(np.pi*LC)) * (np.log(2*LC/aa) + (K1*LC/np.sqrt(A)) - K2)
R2 = (p/(2*np.pi*nR*LR)) * (np.log(4*LR/b) - 1 + (2*K1*Lr/np.sqrt(A)) * (np.sqrt(nR) - 1)**2)
Rm = (p/(np.pi*LC)) * (np.log(2*LC/Lr) + (K1*LC/np.sqrt(A)) - K2 + 1)
Rg = (R1*R2-Rm**2) / (R1+R2-2*Rm)

# Arus Gangguan Ke tanah (If (Amp))
If3 = Vf / Z1

# Luas penampang konduktor (mm2)
Amm2 = If3/(np.sqrt((TCAP*10**-4)/(ts*ar*pr)) * np.log((Ko+Tm)/(Ko+Ta)))


Cs = 1 - ((0.09*(1-(p/ps))) / 2*hs+0.09)
# Estep dan Etouch 50 Kg
Ik50 = 0.116 / np.sqrt(ts)
Es50 = (1000 + 6*Cs*ps) * (Ik50)
Et50 = (1000 + 1.5*Cs*ps) * (0.116/np.sqrt(ts))

# Estep dan Etouch 50 Kg
Ik70 = 0.157 / np.sqrt(ts)
Es70 = (1000 + 6*Cs*ps) * (Ik70)
Et70 = (1000 + 1.5*Cs*ps) * (0.157/np.sqrt(ts))

# Mesh Voltage (Em)
Kh = np.sqrt(1+(h/1))
N  = np.sqrt(n*m)
Ki = 0.644+0.148*N
Io = (3*(E/np.sqrt(3)))/(3*0+(4+4+10)+(10+10+40))
Ig = 0.6 * Io
IG = 1 * Ig
Km = (1 / (2 * np.pi)) * (np.log(D**2/(16*h*d)) + ((D+2*h)**2/(8*D*d)) - (h/(4*d)) + ((Kii/Kh) * np.log(8/(np.pi*(2*N-1)))))
LM = LC + LR
Em = ((p*Km*Ki*IG) / LM) / 100

# Step Voltage (Es)
Ks = (1/np.pi) * ((1/(2*h)) + (1/(D+h)) + ((1/D)*(1-(0.5**(N-2)))))
Es = (p*IG*Ks*Ki) / (0.75*LC + 0.85*LR)

#GPR
GPR = IG * Rg

In [36]:
print(Rg)

0.32710699499800316


In [37]:
LC      = 10*n + 10*m
Q       = m*n - ((m-2)*(n-2))
LR      = Q * 3
L       = LC + LR
Rg      = p * ((1/L) + ((1/np.sqrt(20*A)) * (1+(1/(1+h*np.sqrt(20/A))))))

Rg

0.9445680399570503

In [38]:
m       = 1                     # jumlah konduktor paralel sisi lebar
n       = 1                     # jumlah konduktor paralel sisi panjang
L1      = 10                    # panjang konduktor (m)
L2      = 10                    # lebar konduktor (m)
LC      = L1*n + L2*m           # Total panjang konduktor pentanahan grid (m)
Q       = m*n - ((m-2)*(n-2))   # jumlah konduktor rod
LR      = Q * 3                 # Total panjang konduktor dari batang rod (m)
L       = LC + LR               # Jumlah total panjang konduktor batang rod (m)
Rg      = 0.3324

while Rg <= 10000:
    print(LC, end="")
    print(f"\t||  {LR}", end="")
    print(f"\t||  {Rg}")
    if 0.1 < Rg < 0.3:
        break
    # nR = np.random.randint(1, high=20)
    # h  = np.random.randint(1, high=3)

    LC += 1 # panjang total konduktor Rod (m)
    LR += 2 # panjang total konduktor Mesh (m)

    # RG (Resistansi Pentanahan) Grid-Rod (ohm)
    aa = np.sqrt(a*2*h)
    R1 = (p/(np.pi*LC)) * (np.log(2*LC/aa) + (K1*LC/np.sqrt(A)) - K2)
    R2 = (p/(2*np.pi*nR*LR)) * (np.log(4*LR/b) - 1 + (2*K1*Lr/np.sqrt(A)) * (np.sqrt(nR) - 1)**2)
    Rm = (p/(np.pi*LC)) * (np.log(2*LC/Lr) + (K1*LC/np.sqrt(A)) - K2 + 1)
    Rg = (R1*R2-Rm**2) / (R1+R2-2*Rm)

    # Arus Gangguan Ke tanah (If (Amp))
    If3 = Vf / Z1

    # Luas penampang konduktor (mm2)
    Amm2 = If3/(np.sqrt((TCAP*10**-4)/(ts*ar*pr)) * np.log((Ko+Tm)/(Ko+Ta)))


    Cs = 1 - ((0.09*(1-(p/ps))) / 2*hs+0.09)
    # Estep dan Etouch 50 Kg
    Ik50 = 0.116 / np.sqrt(ts)
    Es50 = (1000 + 6*Cs*ps) * (Ik50)
    Et50 = (1000 + 1.5*Cs*ps) * (0.116/np.sqrt(ts))

    # Estep dan Etouch 50 Kg
    Ik70 = 0.157 / np.sqrt(ts)
    Es70 = (1000 + 6*Cs*ps) * (Ik70)
    Et70 = (1000 + 1.5*Cs*ps) * (0.157/np.sqrt(ts))

    # Mesh Voltage (Em)
    Kh = np.sqrt(1+(h/1))
    N  = np.sqrt(n*m)
    Ki = 0.644+0.148*N
    Io = (3*(E/np.sqrt(3)))/(3*0+(4+4+10)+(10+10+40))
    Ig = 0.6 * Io
    IG = 1 * Ig
    Km = (1 / (2 * np.pi)) * (np.log(D**2/(16*h*d)) + ((D+2*h)**2/(8*D*d)) - (h/(4*d)) + ((Kii/Kh) * np.log(8/(np.pi*(2*N-1)))))
    LM = LC + LR
    Em = ((p*Km*Ki*IG) / LM) / 100

    # Step Voltage (Es)
    Ks = (1/np.pi) * ((1/(2*h)) + (1/(D+h)) + ((1/D)*(1-(0.5**(N-2)))))
    Es = (p*IG*Ks*Ki) / (0.75*LC + 0.85*LR)

    #GPR
    GPR = IG * Rg


print("Final LC :", LC)
print("Final LR :", LR)
print("Final Rg :", Rg)

20	||  0	||  0.3324
21	||  2	||  -0.07411785933103039
22	||  4	||  -0.14448884972307502
23	||  6	||  -0.14856820067438048
24	||  8	||  -0.1345126003558101
25	||  10	||  -0.11475425550558273
26	||  12	||  -0.09371198009057469
27	||  14	||  -0.07318675273335351
28	||  16	||  -0.053940222245043835
29	||  18	||  -0.03627176925586118
30	||  20	||  -0.020261462836077798
31	||  22	||  -0.0058825562740612465
32	||  24	||  0.006942932345728325
33	||  26	||  0.018315569824664233
34	||  28	||  0.028343394436212758
35	||  30	||  0.03713391937931666
36	||  32	||  0.04479000020399818
37	||  34	||  0.05140784427414709
38	||  36	||  0.05707621891328349
39	||  38	||  0.06187632766369429
40	||  40	||  0.06588205054757793
41	||  42	||  0.06916037196987403
42	||  44	||  0.07177189359988773
43	||  46	||  0.07377137282458328
44	||  48	||  0.07520825306969746
45	||  50	||  0.0761271676676357
46	||  52	||  0.07656840816883442
47	||  54	||  0.07656835348272631
48	||  56	||  0.07615985944976013
49	||  58	||  0.

In [41]:
Rg  = 1
nK  = 2     # jumlah konduktor grid
L   = 10    # sisi panjang dan lebar (m)
D   = 0
rod = 0
while Rg <= 10000:
    print(nK, end="")
    print(f"\t||  {D}", end="")
    print(f"\t||  {Rg}")
    if 0 < Rg < 1:
        break
    
    # nK += 1 # panjang total konduktor grid (m)
    nK = 11
    rod += 1

    LC = L*nK + L*nK
    Q  = nK*nK - ((nK-2)*(nK-2))
    LR = Q * 3

    # RG (Resistansi Pentanahan) Grid-Rod (ohm)
    LT = (2 * nK * 10) + rod*3
    D  = 10/(nK - 1) # jarak antar konduktor
    Rg = p * ((1/LT) + ((1/np.sqrt(20*A)) * (1+(1/(1+h*np.sqrt(20/A))))))

    # Arus Gangguan Ke tanah (If (Amp))
    If3 = Vf / Z1

    # Luas penampang konduktor (mm2)
    Amm2 = If3/(np.sqrt((TCAP*10**-4)/(ts*ar*pr)) * np.log((Ko+Tm)/(Ko+Ta)))


    Cs = 1 - ((0.09*(1-(p/ps))) / 2*hs+0.09)
    # Estep dan Etouch 50 Kg
    Ik50 = 0.116 / np.sqrt(ts)
    Es50 = (1000 + 6*Cs*ps) * (Ik50)
    Et50 = (1000 + 1.5*Cs*ps) * (0.116/np.sqrt(ts))

    # Estep dan Etouch 50 Kg
    Ik70 = 0.157 / np.sqrt(ts)
    Es70 = (1000 + 6*Cs*ps) * (Ik70)
    Et70 = (1000 + 1.5*Cs*ps) * (0.157/np.sqrt(ts))

    # Mesh Voltage (Em)
    Kh = np.sqrt(1+(h/1))
    N  = np.sqrt(nK*nK)
    Ki = 0.644+0.148*N
    Io = (3*(E/np.sqrt(3)))/(3*0+(4+4+10)+(10+10+40))
    Ig = 0.6 * Io
    IG = 1 * Ig
    Km = (1 / (2 * np.pi)) * (np.log(D**2/(16*h*d)) + ((D+2*h)**2/(8*D*d)) - (h/(4*d)) + ((Kii/Kh) * np.log(8/(np.pi*(2*N-1)))))
    LM = LC + LR
    Em = ((p*Km*Ki*IG) / LM) / 100

    # Step Voltage (Es)
    Ks = (1/np.pi) * ((1/(2*h)) + (1/(D+h)) + ((1/D)*(1-(0.5**(N-2)))))
    Es = (p*IG*Ks*Ki) / (0.75*LC + 0.85*LR)

    #GPR
    GPR = IG * Rg


print("Final nK :", nK)
print("Final D  :", D)
print("Final rod:", rod)
print("Final LT :", LT)
print("Final Rg :", Rg)

2	||  0	||  1
11	||  1.0	||  0.9831624676568226
Final nK : 11
Final D  : 1.0
Final rod: 1
Final LT : 223
Final Rg : 0.9831624676568226


In [42]:
# LR      = 100               # panjang total konduktor Rod (m)
# LC      = 660               # panjang total konduktor Mesh (m)

print(f"LC                              : {LC} m")
print(f"LR                              : {LR} m")
print(f"Luas penampang konduktor        : {Amm2} mm2")

print("\n============Berat Badan 50 Kg============")
print(f"Arus Fibrasi                    : {Ik50} Amp")
print(f"Tegangan sentuh yang diizinkan  : {Es50} Volt")
print(f"Tegangan langkah yang diizinkan : {Et50} Volt")

print("\n============Berat Badan 70 Kg============")
print(f"Arus Fibrasi                    : {Ik70} Amp")
print(f"Tegangan sentuh yang diizinkan  : {Es70} Volt")
print(f"Tegangan langkah yang diizinkan : {Et70} Volt")

print("\n=============IEEE Grounding==============")
print(f"GPR                             : {GPR} volt")
print(f"Mesh Voltage (Em)               : {Em} volt")
print(f"Step Voltage (Es)               : {Es} volt")
print(f"Rg (resistansi pentanahan) rod  : {Rg} ohm")

LC                              : 220 m
LR                              : 120 m
Luas penampang konduktor        : 83.77059778260876 mm2

============Berat Badan 50 Kg============
Arus Fibrasi                    : 0.116 Amp
Tegangan sentuh yang diizinkan  : 2006.9972000000002 Volt
Tegangan langkah yang diizinkan : 588.7493000000001 Volt

============Berat Badan 70 Kg============
Arus Fibrasi                    : 0.157 Amp
Tegangan sentuh yang diizinkan  : 2716.3669 Volt
Tegangan langkah yang diizinkan : 796.841725 Volt

=============IEEE Grounding==============
GPR                             : 1964.8700147035497 volt
Mesh Voltage (Em)               : 950.9705580321781 volt
Step Voltage (Es)               : 765.8156508603458 volt
Rg (resistansi pentanahan) rod  : 0.9831624676568226 ohm
